In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
from common import *
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [5]:
from dataloaders import readmission_dataset
data = readmission_dataset(args)

INFO - 2019-02-17 13:48:54,295 - Reading Structured data ...
INFO - 2019-02-17 13:48:54,476 - Reading Notes ...
INFO - 2019-02-17 13:49:15,929 - Stratifying ...


In [7]:
from Experiments.training_exps import get_basic_data
train_data, dev_data = get_basic_data(data, truncate=90)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Pos Percentage [0.19738451]


INFO - 2019-02-17 13:49:51,532 - Maximum Sentence Length 721825.000000 , 90 percentile length 18689.400000 ... 
INFO - 2019-02-17 13:49:51,547 - Pos Percentage of remaining data ... 
INFO - 2019-02-17 13:49:51,573 - [0.18685839]


Pos Percentage [0.20937082]


INFO - 2019-02-17 13:49:54,359 - Maximum Sentence Length 588265.000000 , 90 percentile length 19174.500000 ... 
INFO - 2019-02-17 13:49:54,364 - Pos Percentage of remaining data ... 
INFO - 2019-02-17 13:49:54,375 - [0.19723338]


In [27]:
from Experiments.config_exp import CNN_with_attention

In [28]:
config = CNN_with_attention(data, structured=True)

In [29]:
config['model']['attention']['normaliser'] = 'sparsemax'

In [30]:
config['exp_config']['exp_name'] = config['exp_config']['exp_name'].replace('Readmission', 'readmission_sparsemax_test')

In [31]:
from PatientVec.models.Vanilla import ClassificationTrainer as BasicCT
from trainer import Trainer, Evaluator
trainer = Trainer(BasicCT, config, _type=data.metrics_type, display_metrics=True)
trainer.train(train_data, dev_data, n_iters=10, save_on_metric=data.save_on_metric)

evaluator = Evaluator(BasicCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=True)
_ = evaluator.evaluate(dev_data, save_results=True)
print('='*300)

INFO - 2019-02-17 15:37:00,377 - instantiating class <class 'PatientVec.models.Model.Model'> from params {'type': 'seq_classifier_with_attention', 'embedder': {'type': 'token_embedder', 'vocab_size': 32246, 'embed_size': 200, 'embedding_file': './preprocess/Readmission/embedding_matrix.npy'}, 'decoder': {'num_layers': 2, 'hidden_dims': [128, 1], 'activations': ['tanh', 'linear']}, 'predictor': {'type': 'binary'}, 'structured': {'use_structured': True, 'structured_dim': 142}, 'encoder': {'type': 'cnn', 'hidden_size': 64, 'kernel_sizes': [3, 5, 7, 9], 'activation': 'relu'}, 'attention': {'similarity': {'type': 'additive', 'hidden_size': 128}, 'normaliser': 'sparsemax'}} and extras {}
INFO - 2019-02-17 15:37:00,378 - type = seq_classifier_with_attention
INFO - 2019-02-17 15:37:00,380 - instantiating class <class 'PatientVec.models.modules.Embedder.Embedder'> from params {'type': 'token_embedder', 'vocab_size': 32246, 'embed_size': 200, 'embedding_file': './preprocess/Readmission/embedding

Setting Embedding


/media/jainsarthak/data/projects/PatientVec/models/modules/sparsemax.py:54: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  range = torch.range(start=1, end=number_of_logits, device=device).view(1, -1)


{'accuracy': 0.6998363825673063, 'roc_auc': 0.7054027138142497, 'pr_auc': 0.3874697804909929}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.797,0.420,0.7,0.609,0.723
precision,0.870,0.340,0.7,0.605,0.765
recall,0.736,0.552,0.7,0.644,0.700
support,5397.000,1326.000,6723.0,6723.000,6723.000


...

Model not saved on  roc_auc 0.7097021388621296


{'accuracy': 0.6452476572958501, 'roc_auc': 0.6903087883861516, 'pr_auc': 0.3679656325486285}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.747,0.406,0.645,0.576,0.680
precision,0.873,0.303,0.645,0.588,0.761
recall,0.653,0.614,0.645,0.633,0.645
support,5397.000,1326.000,6723.000,6723.000,6723.000


Model not saved on  roc_auc 0.6903087883861516


KeyboardInterrupt: 

In [32]:
outputs = trainer.model.evaluate(dev_data)

In [37]:
n = 1000
print_attn(data.vocab.map_to_words([x for y in dev_data.X[n] for x in y]), outputs['attentions'][n])

''

In [36]:
outputs['attentions'][n]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)